In [1]:
import json
import requests
import os
from datetime import datetime
import zipfile
import tempfile
from collections import defaultdict

In [2]:
JSON_FILE = "./memories_history.json"

In [3]:
# Output folder
output_dir = "snapchat_downloads_renamed"
os.makedirs(output_dir, exist_ok=True)

In [4]:
# Load JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

In [5]:
memories = data['Saved Media']

In [6]:
def extension_from_headers(headers, media_type):
    content_type = headers.get("Content-Type", "").lower()

    if "video/mp4" in content_type:
        return ".mp4"
    if "image/jpeg" in content_type:
        return ".jpg"
    if "image/png" in content_type:
        return ".png"
    if "image/webp" in content_type:
        return ".webp"
    if "application/zip" in content_type:
        return ".zip"

    # fallback if Snapchat does something weird
    if media_type == "Video":
        return ".mp4"
    if media_type == "Image":
        return ".jpg"

    return ""
    
def safe_filename(s):
    return "".join(c for c in s if c.isalnum() or c in "._- ").rstrip()

def download_file(url, out_path, cookies=None):
    r = requests.get(url, stream=True, cookies=cookies)
    r.raise_for_status()
    with open(out_path, "wb") as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return r.headers.get("content-type")

In [7]:
for item in memories:
    url = item["Media Download Url"]
    media_type = item["Media Type"]

    # clean date for filename
    date = datetime.strptime(item["Date"], "%Y-%m-%d %H:%M:%S UTC")
    base_name = date.strftime("%Y-%m-%d_%H%M%S")

    try:
        response = requests.get(url, allow_redirects=True)
        response.raise_for_status()
    except:
        if response.status_code >= 500:
            time.sleep(5)
            try:
                response = requests.get(url, allow_redirects=True)
                response.raise_for_status()
            except:
                print("Could not save: ", filename)
                pass
            

    ext = extension_from_headers(response.headers, media_type)
    filename = os.path.join(output_dir, base_name + ext)
    
    with open(filename, "wb") as f:
        f.write(response.content)

    print("Saved:", filename)


Saved: snapchat_downloads_renamed\2025-12-31_162224.zip
Saved: snapchat_downloads_renamed\2025-12-31_161549.zip
Saved: snapchat_downloads_renamed\2025-12-29_154007.jpg
Saved: snapchat_downloads_renamed\2025-12-28_201420.mp4
Saved: snapchat_downloads_renamed\2025-12-28_195554.jpg
Saved: snapchat_downloads_renamed\2025-12-28_000303.mp4
Saved: snapchat_downloads_renamed\2025-12-27_171140.jpg
Saved: snapchat_downloads_renamed\2025-12-27_163847.jpg
Saved: snapchat_downloads_renamed\2025-12-27_162429.jpg
Saved: snapchat_downloads_renamed\2025-12-21_182956.jpg
Saved: snapchat_downloads_renamed\2025-12-21_135233.jpg
Saved: snapchat_downloads_renamed\2025-12-21_134712.mp4
Saved: snapchat_downloads_renamed\2025-12-21_112725.mp4
Saved: snapchat_downloads_renamed\2025-12-21_112702.mp4
Saved: snapchat_downloads_renamed\2025-12-21_083517.mp4
Saved: snapchat_downloads_renamed\2025-12-21_082203.mp4
Saved: snapchat_downloads_renamed\2025-12-20_140328.mp4
Saved: snapchat_downloads_renamed\2025-12-20_093

Saved: snapchat_downloads_renamed\2025-09-21_171006.jpg
Saved: snapchat_downloads_renamed\2025-09-21_170958.jpg
Saved: snapchat_downloads_renamed\2025-09-21_170935.jpg
Saved: snapchat_downloads_renamed\2025-09-21_091551.jpg
Saved: snapchat_downloads_renamed\2025-09-21_091539.jpg
Saved: snapchat_downloads_renamed\2025-09-16_213046.mp4
Saved: snapchat_downloads_renamed\2025-09-16_170634.jpg
Saved: snapchat_downloads_renamed\2025-09-16_091235.zip
Saved: snapchat_downloads_renamed\2025-09-16_075915.mp4
Saved: snapchat_downloads_renamed\2025-09-16_075906.mp4
Saved: snapchat_downloads_renamed\2025-09-15_230605.zip
Saved: snapchat_downloads_renamed\2025-09-15_094145.zip
Saved: snapchat_downloads_renamed\2025-09-14_130758.jpg
Saved: snapchat_downloads_renamed\2025-09-09_150611.mp4
Saved: snapchat_downloads_renamed\2025-09-08_234812.zip
Saved: snapchat_downloads_renamed\2025-09-08_141338.jpg
Saved: snapchat_downloads_renamed\2025-09-08_112318.mp4
Saved: snapchat_downloads_renamed\2025-09-06_204

HTTPError: 500 Server Error: Internal Server Error for url: https://us-east1-aws.api.snapchat.com/dmd/mm?uid=3a28fa43-6906-47be-b3cc-cc7a5dacfb67&sid=39CCDF17-EA83-41CF-9EEA-249A99FD4B58&mid=39CCDF17-EA83-41CF-9EEA-249A99FD4B58&ts=1768656405664&sig=cdcd9f37df08a99dce9c4daa3324d6c5096200078c825888efdb249395aac69e